In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-in-csv/mnist_test.csv
/kaggle/input/mnist-in-csv/mnist_train.csv
/kaggle/input/mnist-train/mnist_train.csv


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.optim import Adam
import time
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

from sklearn.model_selection import train_test_split

torch_X_train = torch.from_numpy(x_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(x_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets (tensor objects)
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

In [3]:
mnist_train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
mnist_test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

In [4]:
y_train,x_train = mnist_train['label'].values, mnist_train.iloc[:,1:].values
y_test,x_test = mnist_test['label'].values,mnist_test.iloc[:,1:].values

In [5]:
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1- TRAIN_SPLIT
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

torch_X_train = torch.from_numpy(x_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(x_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets (tensor objects)
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

numTrainSamples = int(len(train)*TRAIN_SPLIT)
numValSamples = int(len(train)*VAL_SPLIT)
(trainData,valData) = random_split(train,[numTrainSamples,numValSamples],generator = torch.Generator().manual_seed(42))

# data loader object
train_loader = torch.utils.data.DataLoader(trainData, batch_size = BATCH_SIZE, shuffle = False)
val_loader = torch.utils.data.DataLoader(valData, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

#calculkate steps per epcoh for training and validation set
trainSteps = len(train_loader.dataset)
valSteps = len(val_loader.dataset)


In [15]:
class LeNet(nn.Module):
    def __init__(self,numChannels,classes):
        #call the parent constructor
        super(LeNet,self).__init__()
        #initialise first set of convolutions
        self.conv1 = nn.Conv2d(in_channels = numChannels, out_channels = 20,kernel_size = (5,5))
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size = (2,2),stride = (2,2))
        
        #initialise second set of convolutions
        
        self.conv2 = nn.Conv2d(in_channels = 20,out_channels = 50,kernel_size = (5,5))
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size = (2,2),stride = (2,2))
        
        #initialise first set of FC 
        self.fc1 = nn.Linear(in_features = 800,out_features = 500)
        self.relu3 = nn.ReLU()
        
        #initialise softmax classifier
        self.fc2 = nn.Linear(in_features=500,out_features = classes)
        self.logSoftmax = nn.LogSoftmax(dim = 1)
    def forward(self,x):
        x = x.to(torch.float)
        x = x.view(1,64,28,28)
        print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
    
    # pass the output from the previous layer through the second
    # set of CONV => RELU => POOL layers
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
    
    # flatten the output from the previous layer and pass it
    # through our only set of FC => RELU layers
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
    
    # pass the output to our softmax classifier to get our output
    # predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
    
    # return the output predictions
        return output
    

In [13]:
print('Initialising the lenet model')
model = LeNet(numChannels = 1,classes=10)

#initialising optimizer and loss function
opt = Adam(model.parameters(),lr = INIT_LR)
# cross entropy loss
lossFn = nn.NLLLoss()
#initialising a dictionary to store training history
H = {"train_loss":[],
    "train_acc":[],
    "val_loss":[],
    "val_acc":[]
    }
print('training the network')
# starting out timer
startTime = time.time()

Initialising the lenet model
training the network


In [14]:
for e in range(EPOCHS):
    model.train()
    totalTrainLoss = 0
    totalValLoss = 0
    trainCorrect = 0
    valCorrect = 0
    for(x,y) in train_loader:
        (x,y) = (x.to(device),y.to(device))
        # forward pass
        pred = model(x)
        loss = lossFn(pred,y)
        # zero out the gradients, perform backpropagation and update weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        # add loss to total training losss
        # calculate the number of correct predictions
        totalTrainLoss = totalTrainLoss + loss
        trainCorrect = trainCorrect + (pred.argmax(1)==y).type(torch.float).sum().item()
        

RuntimeError: Given groups=1, weight of size [20, 1, 5, 5], expected input[1, 64, 28, 28] to have 1 channels, but got 64 channels instead

In [ ]:
with torch.no_grad():
    model.eval()
    for (x,y) in valDataLoader:
        (x,y) = (x.to(device),y.to(device))
        pred = model(x)
        totalValLoss = totalValLoss + lossFn(pred,y)
        valCorrect += (pred.argmax(1)==y).type(torch.float).sum().item()

In [ ]:
def create_lenet():
    model = nn.Sequential(
        nn.Conv2d(1, 6, 5, padding=2),
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2),
        nn.Conv2d(6, 16, 5, padding=0),
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2),
        nn.Flatten(),
        nn.Linear(400, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10)
    )
    return model

In [ ]:
plt.figure(figsize = (7,7))
idx = 10
grid_data = x.iloc[idx].values.reshape(28,28)
plt.imshow(grid_data,interpolation = 'none', cmap = 'gray')
plt.show()

In [ ]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    return correct*100./total

In [ ]:
def train(numb_epoch=3, lr=1e-3, device="cpu"):
    accuracies = []
    cnn = create_lenet().to(device)
    cec = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr=lr)
    max_accuracy = 0
    for epoch in range(numb_epoch):
        for images,labels in enumerate(mnist_train):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            pred = cnn(images)
            loss = cec(pred, labels)
            loss.backward()
            optimizer.step()
        accuracy = float(validate(cnn,mnist_test))
        accuracies.append(accuracy)
        if accuracy > max_accuracy:
            best_model = copy.deepcopy(cnn)
            max_accuracy = accuracy
            print("Saving Best Model with Accuracy: ", accuracy)
        print('Epoch:', epoch+1, "Accuracy :", accuracy, '%')
    plt.plot(accuracies)
    return best_model

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
device

In [ ]:
lenet = train(40,device = device)